## Raking of the AllOfUs Data

This notebook performs raking according to David Marker's memo 5.4. It tries the univariate raking model and all models with pairwise interactions and scores each one.

### Setup workspace

Make sure [repo](https://github.gatech.edu/rb230/t10-raking) is checked out to `/home/jupyter/workspaces/duplicateofgtrihealthanalytics`.

In [ ]:
import os
import re
import sys

#### User Parameters

In [ ]:
# set to True to use the imputed income data
IMPUTE_INCOME = True

# four-digit year of the PUMS target data, must already be recoded and collapsed
PUMS_YEAR = 2020

# directory where the source and target recoded input files are located
DATA_DIR = 'data'

# threshold for auto-collapse of bins; set to 0 to disable
AUTO_COLLAPSE_THRESHOLD = 0

# minimum count in any cell of any source marginal distribution prior to raking
MIN_CELL_SIZE = 50

# which model to use for weighting (index into the MODELS list)
# 0 : univariate
# 1 : [45]
# 2 : [35]
MODEL_INDEX = 2

# write result files for each state here
# each state file has the form aou_xx_models.csv
#OUTPUT_DIR = os.path.join('results', 'weighting_{0}'.format(PUMS_YEAR), 'model_{0}'.format(MODEL_INDEX))
OUTPUT_DIR = os.path.join('results', 'weight_as_{0}'.format(PUMS_YEAR))

In [ ]:
print('Output directory: {0}'.format(OUTPUT_DIR))

#### End of User Parameters

In [ ]:
repo_dir = '/home/jupyter/workspaces/duplicateofgtrihealthanalytics/t10-raking'
assert os.path.exists(repo_dir)
assert os.listdir(repo_dir)
sys.path.insert(1, repo_dir)

if not os.path.isdir(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

Ensure that some additional packages are installed.

In [ ]:
%pip install ipfn
%pip install tabulate

In [ ]:
import math
import requests
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from tabulate import tabulate
from collections import Counter, defaultdict

from src import plots, pdf, autocollapse, raking, census_divisions as census

# coding file - ensure that the input files were generated with the selected coding scheme
from src import coding_aou as CODING

### Load the AllOfUs Data

In [ ]:
if IMPUTE_INCOME:
    name_of_file_in_bucket = 'AoU_Impute.csv'
else:
    name_of_file_in_bucket = 'baseline_population_v5.csv'
    
my_bucket = os.getenv('WORKSPACE_BUCKET')
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
baseline = pd.read_csv(name_of_file_in_bucket)

In [ ]:
baseline

In [ ]:
for c in baseline.columns:
    print(c)

### Download the Recoded PUMS USA File

In [ ]:
# recoded PUMS file for all 50 states and DC
PUMS_RECODED_FILE_NAME = 'pums_usa_{0}_recoded_aou.csv'.format(PUMS_YEAR)

Path("data").mkdir(parents=True, exist_ok=True)
#aou_pums = 'data/pums_usa_{0}_recoded_aou.csv'.format(PUMS_YEAR)
aou_pums = os.path.join(DATA_DIR, PUMS_RECODED_FILE_NAME)
with open(aou_pums, 'w') as f:
    if 2019 == PUMS_YEAR:
        res = requests.get('https://gtri.box.com/shared/static/lonqp4hcbdr1wk9anhd7obbqbqxz8vww.csv')
    elif 2020 == PUMS_YEAR:
        res = requests.get('https://gtri.box.com/shared/static/lwgs9nl7b9hw7h4e1niatszvu4oq3ulx.csv')
        
    f.write(res.text)

### Other Parameters

In [ ]:
# Identifiers for the variables to be raked (from the coding file) - also specifies their ordering.
# *** The ordering of the variables affects the raking metric. ***

# for imputed income file
if IMPUTE_INCOME:
    RAKE_DATA = [
        # variable enum,              # recoded source var name,   # recoded PUMS var name
        (CODING.Variables.AGE,        'AGEc',                      'Age'),          
#         (CODING.Variables.RACE_ETH,   'RACE_GROUPING',             'RaceEth'),
#         (CODING.Variables.INSURANCE,  'INSURANCE_GROUPING',        'Insurance'),
#         (CODING.Variables.EDUCATION,  'EDUCATION_GROUPING',        'Education'),
#         (CODING.Variables.INCOME,     'INCOME_NUM',                'Income'),  
        (CODING.Variables.SEX,        'sg',                        'Sex'),        
    ]
else:
    RAKE_DATA = [
        # variable enum,              # recoded source var name,   # recoded PUMS var name
        (CODING.Variables.AGE,        'AGE_GROUPING',              'Age'),          
#         (CODING.Variables.RACE_ETH,   'RACE_GROUPING',             'RaceEth'),
#         (CODING.Variables.INSURANCE,  'INSURANCE_GROUPING',        'Insurance'),
#         (CODING.Variables.EDUCATION,  'EDUCATION_GROUPING',        'Education'),
#         (CODING.Variables.INCOME,     'INCOME_GROUPING',           'Income'),  
        (CODING.Variables.SEX,        'SEX_AT_BIRTH',              'Sex'),        
    ]    

# map of the enumvar to (source, target) colum name tuple
RAKE_MAP = {enumvar:(source_col, target_col) for enumvar, source_col, target_col in RAKE_DATA}
    


# directory where the source and target recoded input files are located
#DATA_DIR = 'data'

# name of the recoded NHIS data
#AOU_RECODED_FILE_NAME = 'synthetic_aou.csv'

# Whether to use weighted PUMS counts, which are required for true population counts (PUMS 'PWGTP' variable).
# Set to None to use an unweighted target population.
PUMS_WEIGHT_COL = 'PWGTP'

# source state two-letter abbreviation or abbreviation for census division (listed below)
# census divisions: DIV_NE, DIV_MA, DIV_ENC, DIV_WNC, DIV_SA, DIV_ESC, DIV_WSC, DIV_M, DIV_P
#SOURCE_STATE = 'DIV_P'

# PUMS target state abbreviation
# (if SOURCE_STATE is a census division, the target state must also be in that division)
#TARGET_STATE = 'AK'

# a weighted version of the source file will be written to DATA_DIR
#OUTPUT_FILE_NAME = 'aou_weighted_{0}_{1}.csv'.format(SOURCE_STATE, TARGET_STATE)

### Setup some data structures

In [ ]:
# map of state abbreviations to PUMS state codes
STATE_CODE_MAP = {
    'AL' : 1,
    'AK' : 2,
    'AZ' : 4,
    'AR' : 5,
    'CA' : 6,
    'CO' : 8,
    'CT' : 9,
    'DE' : 10,
    'DC' : 11,
    'FL' : 12,
    'GA' : 13,
    'HI' : 15,
    'ID' : 16,
    'IL' : 17,
    'IN' : 18,
    'IA' : 19,
    'KS' : 20,
    'KY' : 21,
    'LA' : 22,
    'ME' : 23,
    'MD' : 24,
    'MA' : 25,
    'MI' : 26,
    'MN' : 27,
    'MS' : 28,
    'MO' : 29,
    'MT' : 30,
    'NE' : 31,
    'NV' : 32,
    'NH' : 33,
    'NJ' : 34,
    'NM' : 35,
    'NY' : 36,
    'NC' : 37,
    'ND' : 38,
    'OH' : 39,
    'OK' : 40,
    'OR' : 41,
    'PA' : 42,
    'RI' : 44,
    'SC' : 45,
    'SD' : 46,
    'TN' : 47,
    'TX' : 48,
    'UT' : 49,
    'VT' : 50,
    'VA' : 51,
    'WA' : 53,
    'WV' : 54,
    'WI' : 55,
    'WY' : 56,
}

INV_STATE_CODE_MAP = {v:k for k,v in STATE_CODE_MAP.items()}

# map of state abbreviations to its US Census division
CENSUS_DIVISION_MAP = {
    'AL' : 'DIV_ESC',
    'AK' : 'DIV_P',
    'AZ' : 'DIV_M',
    'AR' : 'DIV_WSC',
    'CA' : 'DIV_P',
    'CO' : 'DIV_M',
    'CT' : 'DIV_NE',
    'DE' : 'DIV_SA',
    'DC' : 'DIV_SA',
    'FL' : 'DIV_SA',
    'GA' : 'DIV_SA',
    'HI' : 'DIV_P',
    'ID' : 'DIV_M',
    'IL' : 'DIV_ENC',
    'IN' : 'DIV_ENC',
    'IA' : 'DIV_WNC',
    'KS' : 'DIV_WNC',
    'KY' : 'DIV_ESC',
    'LA' : 'DIV_WSC',
    'ME' : 'DIV_NE',
    'MD' : 'DIV_SA',
    'MA' : 'DIV_NE',
    'MI' : 'DIV_ENC',
    'MN' : 'DIV_WNC',
    'MS' : 'DIV_ESC',
    'MO' : 'DIV_WNC',
    'MT' : 'DIV_M',
    'NE' : 'DIV_WNC',
    'NV' : 'DIV_M',
    'NH' : 'DIV_NE',
    'NJ' : 'DIV_MA',
    'NM' : 'DIV_M',
    'NY' : 'DIV_MA',
    'NC' : 'DIV_SA',
    'ND' : 'DIV_WNC',
    'OH' : 'DIV_ENC',
    'OK' : 'DIV_WSC',
    'OR' : 'DIV_P',
    'PA' : 'DIV_MA',
    'RI' : 'DIV_NE',
    'SC' : 'DIV_SA',
    'SD' : 'DIV_WNC',
    'TN' : 'DIV_ESC',
    'TX' : 'DIV_WSC',
    'UT' : 'DIV_M',
    'VT' : 'DIV_NE',
    'VA' : 'DIV_SA',
    'WA' : 'DIV_P',
    'WV' : 'DIV_SA',
    'WI' : 'DIV_ENC',
    'WY' : 'DIV_M',
}

# the seventeen states with 1K or more AllOfUs samples
STATES_1K = {
    'CA', 'AZ', 'IL', 'PA', 'NY', 'MA', 'AL',
    'WI', 'MI', 'FL', 'TX', 'GA', 'LA', 'MS',
    'SC', 'TN', 'CT'
}

In [ ]:
# list of enumerated variables to be raked
RAKEVARS = [tup[0] for tup in RAKE_DATA]

# map of variable enum to source file col name
AOU_COL_MAP = {RAKEVARS[i]:RAKE_DATA[i][1] for i in range(len(RAKEVARS))}

# map of variable enum to PUMS file col name
PUMS_COL_MAP = {RAKEVARS[i]:RAKE_DATA[i][2] for i in range(len(RAKEVARS))}

# get the names of the variables, in order
RAKEVAR_NAMES = [CODING.VAR_NAMES[enumvar] for enumvar in RAKEVARS]
RAKEVAR_NAMES

In [ ]:
# choose list of models based on the problem dimension
dim = len(RAKEVARS)

if 2 == dim:
    MODELS = raking.MODELS_2
elif 3 == dim:
    MODELS = raking.MODELS_3
elif 4 == dim:
    MODELS = raking.MODELS_4
elif 5 == dim:
    MODELS = raking.MODELS_5
elif 6 == dim:
    MODELS = raking.MODELS_6
else:
    # just use 1D marginals
    MODELS = [
        [[i] for i in range(dim)],
    ]
    
RAKING_MODEL = MODELS[MODEL_INDEX]
print('Using model "{0}"'.format(RAKING_MODEL))

### Load the PUMS data for the USA

In [ ]:
# recoded PUMS file for the entire USA
TARGET_FILE = os.path.join(DATA_DIR, PUMS_RECODED_FILE_NAME)
print('Target file: {0}'.format(TARGET_FILE))

In [ ]:
print('Loading target file "{0}"...'.format(TARGET_FILE))
pums_usa_df = pd.read_csv(TARGET_FILE)
pums_usa_df

#### Extract PUMS data for the target state

In [ ]:
def get_pums_state_data(pums_usa_df, state_abbrev):
    
    # extract PUMS data for the specified state
    #assert TARGET_STATE in STATE_CODE_MAP
    assert state_abbrev in STATE_CODE_MAP
    state_code = STATE_CODE_MAP[state_abbrev]

    # extract all rows for this state
    pums_df = pums_usa_df.loc[pums_usa_df['ST'] == state_code]

    # keep specified data cols and weight col, drop all others
    keep_cols = [col for col in PUMS_COL_MAP.values()]
    keep_cols.append(PUMS_WEIGHT_COL)
    pums_df = pums_df[keep_cols]
    pums_df = pums_df.reset_index(drop=True)
    return pums_df

### Setup the AllOfUs Source Data

In [ ]:
#raw_source_df = baseline
# extract required cols
cols = [c for c in AOU_COL_MAP.values()]
cols.append('STATE')
cols.append('person_id')
raw_source_df = baseline[cols]

In [ ]:
def get_source_data(raw_source_df, state_abbrev):

    if state_abbrev in STATES_1K:
        # single state
        state_set = {state_abbrev}
    else:
        assert state_abbrev in CENSUS_DIVISION_MAP
        division = CENSUS_DIVISION_MAP[state_abbrev]
        state_tuples = census.CENSUS_DIV_MAP[division]
        state_set = {abbrev for abbrev, code in state_tuples}
        print('States in census division {0}: {1}'.format(division, sorted(list(state_set))))
        
#     if 2 == len(SOURCE_STATE):
#         # single state
#         state_set = {SOURCE_STATE}
#     else:
#         # census division
#         assert SOURCE_STATE in census.CENSUS_DIV_MAP
#         state_tuples = census.CENSUS_DIV_MAP[SOURCE_STATE]
#         state_set = {state_abbrev for state_abbrev, state_code in state_tuples}
#         print('States in census division {0}: {1}'.format(SOURCE_STATE, sorted(list(state_set))))

#         if TARGET_STATE not in state_set:
#             raise ValueError('Target state not in specified census division.')

    # extract data for desired state(s)
    source_df = raw_source_df[raw_source_df['STATE'].isin(state_set)]

    source_df = source_df.reset_index(drop=True)
    print('States in dataframe: {0}'.format(sorted(list(source_df['STATE'].unique()))))
    return source_df

#### Recode the source data

In [ ]:
if IMPUTE_INCOME:
    # for imputed income file
    age_recode  = {
        '18-29':0, # 20-29 => 18_29
        '30-39':1, # 30-39 => 30_39
        '40-49':2, 
        '50-59':3, 
        '60-69':4, 
        '70+'  :5, # 70-79 => 70_plus
        '70+'  :5, # 80-89 => 70_plus
        '70+'  :5, # 90-99 => 70_plus
    }
else:
    age_recode  = {
        '20-29':0, # 20-29 => 18_29
        '30-39':1, # 30-39 => 30_39
        '40-49':2, 
        '50-59':3, 
        '60-69':4, 
        '70-79':5, # 70-79 => 70_plus
        '80-89':5, # 80-89 => 70_plus
        '90-99':5, # 90-99 => 70_plus
    }    

In [ ]:
race_recode = {
    'NH White':0, # NH White => NH White
    'NH Black':1, # NH Black => NH Black
    'NH Asian':2, # NH Asian => NH Asian
    'Hispanic':3, # Hispanic => Hispanic
    'Other'   :4, # Other => Other
}

In [ ]:
sex_recode = {
    'Male'  :0, # Male => Male
    'Female':1, # Female => Female
    'Other' :0, # Other => Male
}

In [ ]:
education_recode = {
    'College Grad'    :0, # College Grad => College Grad
    'Some College'    :1, # Some College => Some College
    'High School Grad':2, # HS Grad => HS Grad
    'Not Grad'        :3, # Not HS Grad => Not HS Grad
    'Missing'         :0, # Missing => College Grad
}

In [ ]:
insurance_recode = {
    'Yes'    :0, # Yes => Yes
    'No'     :1, # No => No
    'Missing':1, # Missing => No
}

In [ ]:
if IMPUTE_INCOME:
    income_recode = {
        1:0, # less than $25K => less than $25K
        2:1, # [$25K, $50K) => [$25K, $50K)
        3:2, # [$50K, $100K) => [$50K, $100K)
        4:3, # $100K or more => $100K or more
    }
else:
    income_recode = {
        '<25K'    :0, # less than $25K => less than $25K
        '25-50K'  :1, # [$25K, $50K) => [$25K, $50K)
        '50-100K:':2, # [$50K, $100K) => [$50K, $100K)
        '>100K'   :3, # $100K or more => $100K or more
        'Missing' :4, # Missing => Missing
    }

In [ ]:
# map each variable to its value map
RECODE_MAP = {
    CODING.Variables.AGE       : age_recode,
    CODING.Variables.RACE_ETH  : race_recode,
    CODING.Variables.SEX       : sex_recode,
    CODING.Variables.EDUCATION : education_recode,
    CODING.Variables.INCOME    : income_recode,
    CODING.Variables.INSURANCE : insurance_recode
}

#### Recode the source data

In [ ]:
def recode_source(source_df):
    for enumvar, tup in RAKE_MAP.items():
        source_col, target_col = tup
        value_map = RECODE_MAP[enumvar]
        recoded_values = source_df[source_col].map(value_map)
        source_df = source_df.assign(**{source_col:recoded_values})
        
    return source_df

#### State-Specific Collapsing

In [ ]:
# COLLAPSE AGE
def collapse_age(source_df):
    source_age_col, target_age_col = RAKE_MAP[CODING.Variables.AGE]
    source_df = CODING.collapse_age(source_df, 'STATE', source_age_col)

    # check

    # Connecticut, state code 9
    check_values = source_df[source_df['STATE']=='CT'][source_age_col].values
    ctr = Counter(check_values)
    assert 0 not in ctr.keys()
    # South Carolina, state code 45
    check_values = source_df[source_df['STATE']=='SC'][source_age_col].values
    ctr = Counter(check_values)
    assert 5 not in ctr.keys()
    # States in the West North Central census division
    check_values = source_df[source_df['STATE'].isin({'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD'})][source_age_col].values
    ctr = Counter(check_values)
    assert 3 not in ctr.keys()
    assert 4 not in ctr.keys()
    assert 5 not in ctr.keys()
    
    return source_df

In [ ]:
# COLLAPSE RACE
def collapse_race(source_df):
    source_race_col, target_race_col = RAKE_MAP[CODING.Variables.RACE_ETH]
    source_df = CODING.collapse_raceeth(source_df, 'STATE', source_race_col)

    # check

    # CT, TN, DIV_ESC, DIV_WSC
    check_values = source_df[source_df['STATE'].isin({'CT', 'TN', 'AL', 'KY', 'MS', \
                                                      'TN', 'AR', 'LA', 'OK', 'TX'})][source_race_col].values
    ctr = Counter(check_values)
    assert 2 not in ctr.keys()
    # South Carolina, state code 45
    check_values = source_df[source_df['STATE']=='SC'][source_race_col].values
    ctr = Counter(check_values)
    assert 2 not in ctr.keys()
    assert 4 not in ctr.keys()
    # LA and MS, codes 22 and 28
    check_values = source_df[source_df['STATE'].isin({'LA', 'MS'})][source_race_col].values
    ctr = Counter(check_values)
    assert 2 not in ctr.keys()
    assert 3 not in ctr.keys()
    
    return source_df

In [ ]:
# COLLAPSE EDUCATION
def collapse_education(source_df):
    source_edu_col, target_edu_col = RAKE_MAP[CODING.Variables.EDUCATION]
    source_df = CODING.collapse_education(source_df, 'STATE', source_edu_col)

    # check

    # DIV_WNC
    check_values = source_df[source_df['STATE'].isin({'IA', 'KS', 'MN', 'MO', 
                                                      'NE', 'ND', 'SD'})][source_edu_col].values
    ctr = Counter(check_values)
    assert 3 not in ctr.keys()
    
    return source_df

In [ ]:
# COLLAPSE INCOME
def collapse_income(source_df):
    source_income_col, target_income_col = RAKE_MAP[CODING.Variables.INCOME]

    source_df = CODING.collapse_income(source_df, 'STATE', source_income_col)

    # CT, MS, TN, and SC
    check_values = source_df[source_df['STATE'].isin({'CT', 'MS', 'TN', 'SC'})][source_income_col].values
    ctr = Counter(check_values)
    assert 3 not in ctr.keys()
    # DIV_WNC
    check_values = source_df[source_df['STATE'].isin({'IA', 'KS', 'MN', 'MO', 
                                                      'NE', 'ND', 'SD'})][source_income_col].values
    ctr = Counter(check_values)
    assert 1 not in ctr.keys()
    
    return source_df

In [ ]:
def collapse_variables(df):
    
    if CODING.Variables.AGE in RAKE_MAP:
        df = collapse_age(df)
    if CODING.Variables.RACE_ETH in RAKE_MAP:
        df = collapse_race(df)
    if CODING.Variables.EDUCATION in RAKE_MAP:
        df = collapse_education(df)
    if CODING.Variables.INCOME in RAKE_MAP:
        df = collapse_income(df)
        
    return df

#### Automatic Collapsing

Perform auto-collapsing based on bin counts. Bins with counts below a parameterized threshold are collapsed into an adjacent bin.

In [ ]:
# ORDERED_VARIABLES = {CODING.Variables.AGE, CODING.Variables.EDUCATION, CODING.Variables.INCOME}

# # South Carolina changes the "other" bin for RaceEth
# AUTO_COLLAPSE_RACE_OTHER = CODING.RaceEth.OTHER.value
# if 'SC' == SOURCE_STATE and 'SC' == TARGET_STATE:
#     AUTO_COLLAPSE_RACE_OTHER = 0

# for enumvar, source_col, target_col in RAKE_DATA:
    
#     # print distributions prior to auto-collapse
#     print('Variable: {0}'.format(enumvar))
#     print('BEFORE: ')
#     ctr = Counter(source_df[source_col].values)
#     for k in sorted(ctr.keys()):
#         print('{0} => {1}'.format(k, ctr[k]))
#     ctr = Counter(pums_df[target_col].values)
#     for k in sorted(ctr.keys()):
#         print('\t{0} => {1}'.format(k, ctr[k]))
#     print()    
    
#     if enumvar in ORDERED_VARIABLES:
                
#         source_df, change_list = autocollapse.full_collapse_ordered(source_df,
#                                                                     source_col,
#                                                                     AUTO_COLLAPSE_THRESHOLD)
#         pums_df = autocollapse.collapse_from_changelist(pums_df, target_col, change_list)
        
#         print('\tchange_list {0}: {1}'.format(enumvar, change_list)) 
        
#     elif enumvar == CODING.Variables.RACE_ETH:
#         # unordered
#         source_df, change_list = autocollapse.full_collapse_unordered(source_df,
#                                                                       source_col,
#                                                                       threshold = AUTO_COLLAPSE_THRESHOLD,
#                                                                       other_col = AUTO_COLLAPSE_RACE_OTHER)
#         pums_df = autocollapse.collapse_from_changelist(pums_df, target_col, change_list)
        
#         print('\tchange_list {0}: {1}'.format(enumvar, change_list)) 
        
#     else:
#         # TBD for insurance, sex
#         pass
        
#     # print distributions after auto-collapse
#     print('AFTER: ')
#     ctr = Counter(source_df[source_col].values)
#     for k in sorted(ctr.keys()):
#         print('{0} => {1}'.format(k, ctr[k]))
#     ctr = Counter(pums_df[target_col].values)
#     for k in sorted(ctr.keys()):
#         print('\t{0} => {1}'.format(k, ctr[k]))
#     print()

#### Build target sample arrays

In [ ]:
def get_target_samples(pums_df):
    # get the names of the target columns in the same order as the RAKEVARS array
    ordered_target_cols = [PUMS_COL_MAP[enumvar] for enumvar in RAKEVARS]
    print('Target columns, in order: {0}\n'.format(ordered_target_cols))

    # samples taken in order of the variables in VARIABLES
    TARGET_SAMPLES = [np.array(pums_df[col].values) for col in ordered_target_cols]

    # set TARGET_WEIGHTS to None to rake to an unweighted target population
    if PUMS_WEIGHT_COL is not None:
        TARGET_WEIGHTS = np.array(pums_df[PUMS_WEIGHT_COL].values)
        TARGET_POPULATION = np.sum(TARGET_WEIGHTS)
        print('Target population (weighted): {0}'.format(TARGET_POPULATION))
    else:
        TARGET_WEIGHTS = None
        TARGET_POPULATION = len(TARGET_SAMPLES[0])    
        print('Target population (unweighted): {0}'.format(TARGET_POPULATION))
        
    return TARGET_SAMPLES, TARGET_WEIGHTS, TARGET_POPULATION

#### Build source sample arrays

In [ ]:
def get_source_samples(source_df):
    # get the names of the source dataframe columns in the order matching the variables
    ordered_source_cols = [AOU_COL_MAP[enumvar] for enumvar in RAKEVARS]
    print('Source columns, in order: {0}\n'.format(ordered_source_cols))

    # build a list of np.arrays containing the data for each col
    SOURCE_SAMPLES = [np.array(source_df[col].values) for col in ordered_source_cols]

    print('Sample population: {0}'.format(len(SOURCE_SAMPLES[0])))
    
    return SOURCE_SAMPLES

#### Compute the number of bins required to hold each variable

Categorical values for a given variable are **assumed** to be a contiguous block of integers starting at 0.

In [ ]:
# bin counts in order of the variables
BIN_COUNTS = []

for enumvar in RAKEVARS:
    BIN_COUNTS.append(CODING.BIN_COUNTS[enumvar])

# maximum-length variable name, used for prettyprinting
maxlen = max([len(var_name) for var_name in RAKEVAR_NAMES])

print('Bin counts: ')
for i in range(len(RAKEVAR_NAMES)):
    print('{0:>{2}} : {1}'.format(RAKEVAR_NAMES[i], BIN_COUNTS[i], maxlen))

In [ ]:
# useful plotting function
def plot_pdfs(source_unraked_pdfs, source_raked_pdfs, target_pdfs, source_state, target_state):
    
    # display precision
    P = 5
    
    num_pdfs = len(source_unraked_pdfs)
    assert len(source_raked_pdfs) == len(target_pdfs) == num_pdfs
    
    for q in range(0, num_pdfs):
        plots.triple_histogram_from_pdfs('{0} ({1}-{2})'.format(RAKEVAR_NAMES[q], source_state, target_state), 
                                         source_unraked_pdfs[q], source_raked_pdfs[q], target_pdfs[q],
                                         labels=['Unraked Source', 'Raked Source', 'Target'])
        print('Unraked source PDF : {0}'.format(np.array_str(source_unraked_pdfs[q], precision=P)))
        print('  Raked source PDF : {0}'.format(np.array_str(source_raked_pdfs[q],   precision=P)))
        print('        Target PDF : {0}'.format(np.array_str(target_pdfs[q],         precision=P)))

## Raking in N Dimensions

#### Search for best model

In [ ]:
def score1(weights_m, sample_count):
    """
    The original variability score.
    """
    
    # max weight
    wmax = np.max(weights_m)
    
    # 99th percentile weight
    w99 = np.percentile(weights_m, 99)
    
    # find all weights >= w99
    samples = []
    for w in weights_m:
        if w >= w99:
            samples.append(w)
    
    # std deviation of these weights
    std = np.std(samples)
    
    # the score is the product of the std deviation and the normalized max weight
    # (idea is that a model that minimizes both is better)
    score = (wmax / sample_count)  * std
    return score

In [ ]:
def score2(weights_m, avg_wt):
    """
    David Marker's modification of the variability score.
    """
    
    # will sum all weights >= this fraction * max_wt
    MAX_WT_FRAC = 0.9
    
    # maximum weight for this model
    wmax = np.max(weights_m)
    
    # cutoff value; compute the sum of all weights >= this cutoff
    w_cutoff = MAX_WT_FRAC * wmax
    
    samples = []
    for w in weights_m:
        if w >= w_cutoff:
            samples.append(w)
            
    w_sum = np.sum(samples)
    
    # w_sum represents this percent of the weighted total population
    pct = w_sum / TARGET_POPULATION
     
    # the score is the product of the max wt and the fraction above the cutoff, divided by the avg weight
    score = wmax * pct / avg_wt
    return score

In [ ]:
def score_results(data):

    sample_count = len(SOURCE_SAMPLES[0])
    avg_wt = TARGET_POPULATION / sample_count

    # compute a score for each model
    scores = []
    for wmax, smallest_cell, model_index in data:
        model = MODELS[model_index]
        weights_m = weight_map[model_index]
        score = score2(weights_m, avg_wt)
        scores.append(score)
        
    return scores

In [ ]:
for state_index, state_abbrev in enumerate(STATE_CODE_MAP):
 
    # get target dataframe
    pums_df = get_pums_state_data(pums_usa_df, state_abbrev)

    # get source dataframe (could be multiple states)
    source_df = get_source_data(raw_source_df, state_abbrev)

    # recode the AllOfUs data to the scheme in the CODING file
    # (PUMS has already been recoded to reduce the runtime)
    source_df = recode_source(source_df)

    # collapse the AllOfUs variables
    # (PUMS has already been collapsed to reduce the runtime)
    source_df = collapse_variables(source_df)

    # get source and target sample arrays
    TARGET_SAMPLES, TARGET_WEIGHTS, TARGET_POPULATION = get_target_samples(pums_df)
    SOURCE_SAMPLES = get_source_samples(source_df)
        
    raking_result_tuple = raking.rake(RAKING_MODEL,
                                      SOURCE_SAMPLES,
                                      TARGET_SAMPLES,
                                      TARGET_WEIGHTS,
                                      BIN_COUNTS,
                                      TARGET_POPULATION)

    if raking_result_tuple is None:
        print('\n*** No acceptable raking model was found. ***')
    else:
        weights, unraked, target, raked, fnorm, smallest_cell = raking_result_tuple
        # compute PDFs
        target_pdfs = {}
        source_raked_pdfs = {}
        source_unraked_pdfs = {}

        # target pdfs
        for q in range(len(RAKEVARS)):
            target_pdfs[q] = pdf.to_pdf(BIN_COUNTS[q], TARGET_SAMPLES[q], weights=TARGET_WEIGHTS)

        # unraked source pdfs
        for q in range(len(RAKEVARS)):
            source_unraked_pdfs[q] = pdf.to_pdf(BIN_COUNTS[q], SOURCE_SAMPLES[q], weights=None)

        # raked source pdfs
        for q in range(len(RAKEVARS)):
            source_raked_pdfs[q] = pdf.to_pdf(BIN_COUNTS[q], SOURCE_SAMPLES[q], weights=weights)

        print('\nPDF diffs after raking for model {0}:\n'.format(RAKING_MODEL))   

        # diff between raked and target pdfs
        diffs = []
        for q in range(len(RAKEVARS)):
            diff = source_raked_pdfs[q] - target_pdfs[q]
            diffs.append(diff)
            print('{0:>{2}} : {1}'.format(RAKEVAR_NAMES[q], 
                                          np.array_str(diff, precision=5, suppress_small=True),
                                          maxlen))

        # check the diff vectors for the presence of any diff >= THRESHOLD
        all_ok = True
        THRESHOLD = 0.001
        for diff_vector in diffs:
            if np.any(diff_vector > THRESHOLD):
                all_ok = False

        # sum of the weights
        sum_of_weights = np.sum(weights)
        print()
        print('        Max weight : {0:.3f}'.format(np.max(weights)))
        print('Sum of the weights : {0:.3f}'.format(sum_of_weights))
        print('  Population total : {0:.3f}'.format(TARGET_POPULATION))
        print('        Difference : {0:.3f}'.format(abs(TARGET_POPULATION - sum_of_weights)))
        print('\nRaked PDFs differ from target PDFs by less than {0}: {1}'.format(int(THRESHOLD), all_ok))        
        
        # insert a column for the weights into the source dataframe
        final_df = source_df.assign(Weight = weights)
        
        # construct output file name from the source file name
        if IMPUTE_INCOME:
            output_file = '{0}_imputed.csv'.format(state_abbrev.lower())
        else:
            output_file = '{0}.csv'.format(state_abbrev.lower())
        output_filepath = os.path.join(OUTPUT_DIR, output_file)

        # write output file with only the person_id and the weight
        final_df[['person_id', 'Weight']].to_csv(output_filepath, index=False)
        print('Wrote file "{0}".'.format(output_filepath))

#### Score and rank the results

#### Plot weight components

#### Compute percentage of population in upper weight components

### Compute PDFs for All Variables and Compare with Target


Run the best model again, compute PDFs for all variables, and check for zero diffs with the target.

### Plot 1D Distributions Before and After Raking

#### Print Data on Distributions

### Write out weighted samples

### Check zero-weight tuples

### Find extreme-weighted individuals

### Find PUMS population for specific cells

### Generate MaxWt Table

In [ ]:
def get_max_wt(filepath):
    
    max_wt = 0.0
    with open(filepath, 'rt') as infile:
        for lineno, line in enumerate(infile):
            if 0 == lineno:
                continue
            text = line.strip()
            person_id, weight = text.split(',')
            weight = float(weight)
            if weight > max_wt:
                max_wt = weight
    return max_wt

In [ ]:
FOLDERS = ['weighting_2019', 'weight_as_2019', 'weighting_2020', 'weight_as_2020']

data_2019 = []
folder = os.path.join('results', 'weighting_2019', 'model_2')
for f in os.listdir(folder):
    # get the state abbrev
    state_abbrev = f[6:8]
    filepath = os.path.join(folder, f)
    max_wt_2019 = get_max_wt(filepath)
    data_2019.append( (state_abbrev, max_wt_2019))
data_2019 = sorted(data_2019, key=lambda x: x[0])

data_2020 = []
folder = os.path.join('results', 'weighting_2020', 'model_2')
for f in os.listdir(folder):
    state_abbrev = f[6:8]
    filepath = os.path.join(folder, f)
    max_wt_2020 = get_max_wt(filepath)
    data_2020.append( (state_abbrev,  max_wt_2020))
data_2020 = sorted(data_2020, key=lambda x: x[0])

In [ ]:
data_2_2019 = []
folder = os.path.join('results', 'weight_as_2019')
for f in os.listdir(folder):
    state_abbrev = f[0:2]
    filepath = os.path.join(folder, f)
    max_wt_2019 = get_max_wt(filepath)
    data_2_2019.append( (state_abbrev, max_wt_2019))
data_2_2019 = sorted(data_2_2019, key=lambda x: x[0])

data_2_2020 = []
folder = os.path.join('results', 'weight_as_2020')
for f in os.listdir(folder):
    state_abbrev = f[0:2]
    filepath = os.path.join(folder, f)
    max_wt_2020 = get_max_wt(filepath)
    data_2_2020.append( (state_abbrev, max_wt_2020))
data_2_2020 = sorted(data_2_2020, key=lambda x: x[0])

In [ ]:
assert len(data_2019) == len(data_2_2019) == len(data_2020) == len(data_2_2020)

In [ ]:
data = []
for i, tup in enumerate(data_2019):
    state = tup[0]
    wt_2019 = tup[1]
    wt_2020 = data_2020[i][1]
    wt_as_2019 = data_2_2019[i][1]
    wt_as_2020 = data_2_2020[i][1]
    data.append( (state, wt_2019, wt_as_2019, wt_2020, wt_as_2020))
    #print('{0} : {1}, {2}, {3}, {4}'.format(state, wt_2019, wt_as_2019, wt_2020, wt_as_2020))
    
df = pd.DataFrame(data, columns=['State', 'MAX_WT 2019', 'MAX_WTAS 2019', 'MAX_WT 2020', 'MAX_WTAS 2020'])
df

In [ ]:
#'State', 'MAX_WT 2019', 'MAX_WTAS 2019', 'MAX_WT 2020', 'MAX_WTAS 2020']

# print as csv data
header = ','.join(df.columns)
print(header)
for index, row in df.iterrows():
    state = row['State']
    max_wt_2019 = str(row['MAX_WT 2019'])
    max_wtas_2019 = str(row['MAX_WTAS 2019'])
    max_wt_2020 = str(row['MAX_WT 2020'])
    max_wtas_2020 = str(row['MAX_WTAS 2020'])
    line = ','.join([state, max_wt_2019, max_wtas_2019, max_wt_2020, max_wtas_2020])
    print(line)